# Optimization and Gradient Descent Lecture

F. Burkholder (credit T. Heilman and A. Richards) with edits by Jessica Curley 

## Objectives GO HERE
- Review supervised learning topics
- Introduce optimization methods 
- Learn algorithms to implement the Gradient Descent Algorithm and understand different convergent criteria

### Review 
- What is supervised learning?
- Name two examples of supervised learning we've learned so far
- How do we interpret the coefficients of linear regression? Of logistic regression?
- What is a cost function associated with linear regression?
- What are two types of regularized regression?
- How does each type of regularization handle the coefficients for highly correlated features?

### Connection and Motivation
Name a function you are trying to optimize with **logistic regression**. Is this something we want to maximize or minimize?

## What is Gradient Descent?
**Gradient Descent is not a machine learning model** - it's a way of updating the coefficients of a model, given true and predicted target values (from the model that uses these coefficients), and a **cost function** that quantifies how different the the predicted and true values are across the training data.

Gradient Descent is one way of performing [mathematical optimization.](https://en.wikipedia.org/wiki/Mathematical_optimization)

### Why Do We Care?
The point of this lecture is expose you to what's happening behind the scenes when you are calling `.fit` on some of sklearn's machine learning models to determine model coefficients (a.k.a. $\beta$, $\theta$, parameters, weights)  

This same process also helps determine weights in a neural network - so we will revisit this topic when talk neural nets.  



### Some Terminology
- Cost function: $ J(\theta) $
- Likelihood function: $ L(\theta) $
- Hypothesis function (the function that predicts $y$ from $x$): $ h_\theta(x_i)$    
- Parameters / coefficients / weights : $\theta$ or $\beta$ or *w*
- Learning rate $\alpha$ or $\lambda$

As usual, you will encounter lots of different terms and symbols to represent the same things regarding gradient descent.

# Optimization

Mathematical optimization includes finding "best available" values of some **objective function** given a defined domain (or input).

Example "real world" objective functions:
* The time spent on preparations before leaving your house so that you can arrive at work on time
* Deciding what to do now so that you can retire early
* Finding the maximum or minimum of some mathematical function

In an optimization problem you decide whether you are looking for the objective function's **maximum** or **minimum**. 


## Optimization in machine learning

In [supervised learning](https://en.wikipedia.org/wiki/Supervised_learning) our task is to choose a model, including its coefficients, that will train on existing data and then perform well on unseen data.

Here are two examples of objective functions in machine learning:

### Linear Regression

- Objective function to **minimize** = MSE (this is also known as the cost function here)

Find coefficients $\theta$ that **minimize** the cost function (mean squared error or mean squared residual)

$$ J(\theta) = \frac{1}{N}\sum_{i=1}^N (y_i - h_\theta(x_i))^2 $$  

$h_\theta(x)$ are the predictions of the model in this case, otherwise known as $\hat{y}$ or "y-hat"

### Logistic Regression

- Objective function to **maximize** is the log likelihood 

In the case of binary classification (label is 0 or 1), find coefficients $\theta$ that **maximize** the likelihood of classifying the true values correctly. We want to maximize the *true positive rate* or *sensitivity*

$$  L(\theta) = \sum_{i=1}^N (y_i \ln h_\theta (x_i) + (1- y_i)\ln(1- (h_\theta (x_i))) $$

Why the **maximum**?  Let's review what the natural logarithm looks like for values < 1:   [interactive chart](https://www.google.com/search?client=ubuntu&channel=fs&q=graph+of+natural+logarithm&ie=utf-8&oe=utf-8)

Note that $h_\theta(x)$ are the probability predictions of the model, all ranging from 0 to 1.  A threshold is used with these probabilities to make the prediction $\hat{y}$.  

Let's see how this loss function behaves on 2 datapoints when given a bad model (where the coefficients give bad predictions and a large loss) and a better model (where the coefficients give better prediction and a lower loss).  

Recall: $y_i$ is the true value of the target ($i$ for the *ith* row), and $h_\theta (x_i)$ is the probability of $x_i$ being in the positive class (1) given the features $x_i$ in a logistic regression model of the form:

$$\ln \left( \frac{h_\theta (x)}{1 - h_\theta (x)} \right) = \beta_0 + \beta_1 \cdot X_1 + \beta_2 \cdot X_2 + ... \beta_p \cdot X_p$$


Model A: The "worse" model. Note the probabilities of being in the positive class $h_\theta (x_i)$ given the true value ($y_i$=1) are pretty bad.

|$$y_i$$   |  $$h_\theta (x_i)$$  |$$y_i \ln h_\theta (x_i)$$ | $$(1- y_i)\ln(1- h_\theta (x_i))$$ |$$L(\theta)$$ |
|:---:|:---:|:---:|:---:|:---:|
|0  |0.9 |  0 | -2.3  | -2.3  |
|1  |0.2 | -1.6 | 0   | -1.6  |
|   |    |      |     | -3.9  |  

Model B: A "better" model. It has different coefficients than Model A and better probabilites 

|$$y_i$$   |  $$h_\theta (x_i)$$  |$$y_i \ln h_\theta (x_i)$$ | $$(1- y_i)\ln(1- h_\theta (x_i))$$ |$$L(\theta)$$ |
|:---:|:---:|:---:|:---:|:---:|
|0  |0.2 |  0 | -0.2  | -0.2  |
|1  |0.9 | -0.1 | 0   | -0.1  |
|   |    |      |     | -0.3  |  

Note that the cost or loss is "less" (in this case, maximized) for the model with better coefficients.

We can't always get the best coefficients by solving for the derivative of the objective function our model. Instead, we can use another process that also uses calculus to find the coefficients ...

### Gradient Descent - A Way to Find the Coefficients

[Gradient descent](https://en.wikipedia.org/wiki/Gradient_descent) is a first-order iterative optimization algorithm for finding the minimum of a function.

To find a local minimum, one takes steps proportional to the negative of the gradient (or its approximation) of the function at the current point.

$$ \beta_{new} = \beta_{old} - \alpha \cdot \frac{\nabla J}{\partial \beta}$$

$ \beta_{new} $ is the new estimate of the model coefficients  
$ \beta_{old} $ is the old estimate of the model coefficients  
$ \alpha $ is the learning rate, the tunable parameter that adjusts how much of a step is taken in the direction of the gradient  
$ \frac{\nabla J}{\partial \beta} $ is the gradient, a measure of how much the cost function is increasing with respect to each of the coefficients

![title](images/gradient_desc_img.jpeg)

## Gradient Descent in one dimension

Let's start with a simplified example, a cost function of $J(x) = 3x^2$  

$J$ is the cost, $x$ is the feature/coefficient/parameter that we are trying to update to minimize $J$. 

Gradient descent requires: 
* A cost function, $J$
* The gradient of the cost function, in this case $ \frac{\nabla J}{\partial x} $

In [ ]:
# imports
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams.update({'font.size': 14})
np.set_printoptions(suppress=True)

In [ ]:
def cost_func(x):
    return 3*x**2

def grad_cost_func(x):
    return 6*x

Since this function is differentiable, we know from calculus that we can find the minimum of this function at x = 0. 

For demonstation purposes, we pretend that we don't know this, and we will start with a guess of x = 5 for the minimum.

Let's plot this, for a visual...

In [ ]:
# keeping our guesses in a list for reasons that will become clear later....
guesses = [5]

def plot_cost(x_guess):
    fig = plt.figure(figsize=(8,6))
    ax = fig.add_subplot(111) 
    x = np.linspace(-6, 6)
    y = cost_func(x)
    ax.plot(x, y)
    y_guess = [cost_func(xg) for xg in x_guess]
    ax.plot(x_guess, y_guess, 'ro')
    ax.set_xlabel('x')
    ax.set_ylabel('J')
    
    labels = ['step {}'.format(i) for i in range(len(x_guess))]
    for label, x, y in zip(labels, x_guess, y_guess):
        plt.annotate(
            label,
            xy=(x, y), xytext=(-20, 20),
            textcoords='offset points', ha='right', va='bottom',
            bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.5),
            arrowprops=dict(arrowstyle = '->', connectionstyle='arc3,rad=0'))

   
plot_cost(guesses)

In this one-dimensional case, we can visually inspect and see that we are pretty far from the minimum! 

Let's compute the gradient at this point to update our "guess" for $x$ 

In [ ]:
#recall grad_cost_funcion in this case is 6x, or the derivative of the cost functoin 
grad = grad_cost_func(guesses[-1])
print(grad)

The gradient is huge at our guess! (Remember, this is really just the derivative of the cost function at $x=5$). If we simply adjusted by that amount, we would overshoot the minimum by far! This is why we use the learning rate $\alpha$ or $\lambda$ to adjust our guess by small steps. Let's try a learning rate of .05, so we don't go too far. Since we are doing gradient DESCENT, we should subtract the gradient from our guess. 

In [ ]:
learning_rate = .05 # sometimes referred to as alpha or lambda

guess_update = guesses[-1] - learning_rate*grad
# let's keep our guesses in a list...
guesses.append(guess_update)

print(guesses)

Plot including our new guess...

In [ ]:
plot_cost(guesses)

That's a lot closer. Let's do this three more times and see where we land...

In [ ]:
for _ in range(3):
    grad = grad_cost_func(guesses[-1])
    guess_update = guesses[-1] - learning_rate*grad
    guesses.append(guess_update)

plot_cost(guesses)

Note that as we get closer to the minimum, the gradient gets smaller and so the guesses step more slowly. This is a good thing - a natural property that makes it harder for us to overshoot!

Let's see if 5 more times does the job.

In [ ]:
for _ in range(5):
    grad = grad_cost_func(guesses[-1])
    guess_update = guesses[-1] - learning_rate*grad
    guesses.append(guess_update)

plot_cost(guesses)

looks pretty close, let's inspect the list to see

In [ ]:
print("#\tguess")
for i, guess in enumerate(guesses,1):
    print("{0}\t{1:0.3f}".format(i, guess))

Didn't quite get to the minimum, but with more steps it would.

This was a simple example where there was only one coefficient to find (there was just one feature).  For most machine learning models, there are multiple features and therefore coefficients to find for each feature.

In this case you take a multi-dimensional derivative called the **gradient**.

##  Gradient Formula
* The gradient is the multivariate analogue (dealing with multiple independent variables) of the derivative. 

$$ \nabla f = \sum_{i=1}^P\frac{\partial f}{\partial x_i} \vec{e_i}$$

Where:  
$ \nabla f$ is the gradient of function $f$  

$ \sum_{i=1}^P$ is the sum over all the predictors (columns) $P$  

$ \frac{\partial f}{\partial x_i}$ is the partial derivative of $f$ with respect to predictor $x_i$  

$ e_i$ indicates in the direction of the predictor $x_i$

Simple example:  Say there were columns $x$, $y$, and $z$ in our X (feature) array, 
defined to go in directions $\vec{i}, \vec{j}, \vec{k}$.  
So $$ \nabla f = \frac{\partial f}{\partial x} \vec{i} + \frac{\partial f}{\partial y} \vec{j}+ \frac{\partial f}{\partial z} \vec{k}$$

Say $$ f(x,y,z) = 2x + 3y^{2} - sin(3z) $$

The gradient is:  

$$ \nabla f = 2\vec{i} + 6y\vec{j} - 3cos(3z)\vec{k} $$


## The Flavors of Gradient Descent
- **Batch gradient descent** - computes the gradient of the cost function with respect to $\theta$ for the entire dataset 
    -  accumulates the gradient for all rows of data first, and then make an update.
- **Mini-batch gradient descent** - performs an update for every mini-batch of training examples
    - the typical algorithm for neural network training
    - choosing a learning rate can be tough
- **Stochastic gradient descent (SGD)** - performs gradient descent for each training example in x along with its corresponding y
    - updates are performed with each training example in $x$
    - in practice converges faster than batch gd
    - Here is the pseudocode, where n is the number of training examples:
    ```python
    Randomly shuffle examples in the training set.
    Repeat until step size is sufficiently small:
        for i = 1, 2, ... n:
            β <- β - α * gradient for x_i
    ```

## Morning Assignment Preview

In your assignment today you'll be working through the gradient descent algorithm for logistic regression.  

In this notebook we'll work through an example using linear regression.

### First, recall the objective functions

#### Linear Regression

We want to find coefficients $\theta$ that *minimize* the mean squared residual:

$$ J(\theta) = \frac{1}{N}\sum_{i=1}^N (y_i - h_\theta(x_i))^2 $$

#### Logistic Regression

We want to find coefficients $\theta$ that *maximize* the likelihood of the classifying the true values correctly.

$$  L(\theta) = \sum_{i=1}^N (y_i \ln h_\theta (x_i) + (1- y_i)\ln(1- (h_\theta (x_i))) $$

$ h_\theta (x_i) $ is the predicted probability of the positive class.

To use gradient **descent** we would like to find a minimum instead of a maximum.  So we multiply the maximum likelihood by -1 to change the objective function in to a cost function.

$$  J(\theta) = -\sum_{i=1}^N (y_i \ln h_\theta (x_i) + (1- y_i)\ln(1- (h_\theta (x_i))) $$

In this form, this is equation is the [binary cross-entropy loss function](https://towardsdatascience.com/understanding-binary-cross-entropy-log-loss-a-visual-explanation-a3ac6025181a).  When it's extrapolated to more than two classes it's simply called [categorical cross entropy](https://machinelearningmastery.com/cross-entropy-for-machine-learning/).


### Linear Regression Using Gradient Descent Demo
First, instantiate a small "dataset" X, with 2 features and 10 rows, and **true** beta coefficients (that gradient descent will determine later in the notebook). 

In [ ]:
np.random.seed(1) # set random seed
nrows = 10
const = np.ones((nrows, 1))
x1 = np.random.random((nrows, 1)) # nrows number of random floats 
X = np.hstack([const, x1]) # stack arrays horizontally, or column-wise 

# define true betas that will be used to calculate y
# the goal will be to converge on the true betas using X, y, and gradient descent
true_betas = np.array([3, 4]).reshape(-1, 1) # turns it into a column vector 
print("The betas (or weights, or theta in cost functions above) gradient descent will try to find are:")
print(true_betas)

In [ ]:
X

Typically we know our Xs and $y$s, but are actually trying to solve for the best values of beta. In this example the true beta values are used calculate $y$, and then X and $y$ are used in gradient descent to try to determine what the beta values were.

**But don't we already know the correct answer for our coefficients?** Yes- this is just a good way to practice our algorithm by checking it against the correct answer!

In [ ]:
y = np.dot(X,true_betas).reshape(-1, 1) # keep this as a column vector so linear algebra goes smoothly
print("X array:")
print(np.around(X,2))
print("\ny array:")
print(np.around(y,2))

In [ ]:
# note that np.around did not change the values in the arrays
# It just made it nice to display
y

Yay!

## Practice

### Steps for Gradient Descent

**1. Compute gradient of the cost function.**  
Compute the gradient (derivative) of this below. You may find it helpful to first think of it as just one row of data.

$$ J_i(\beta) = \frac{1}{2}((y_i - x_i \cdot \beta))^2 $$
where  

$ y_i $ is the ith y value (a scalar)  
$ x_i $ is a row vector of two values in this case (X has two columns): [$x_{i,0}$, $x_{i,1}$]  
$ \beta $ is a column vector of the coefficents: $[\beta_0, \beta_1]^T$  
$ x_i \cdot \beta $ is the dot product of these two vectors, yielding a scalar


Work it out on your own...

In [ ]:
def gradient_one_coefficient_one_row(y_i, x_i, beta_guess, coeff_index):
    '''
    Parameters
    ----------
    y_i: one y value that is the target of the row x_i
    x_i: one row of X
    beta_guess: guess values for the 2 coefficients (2 row, 1 column vector)
    coeff_index: index (0 or 1) corresponding to which coefficient gradient applies to
    
    Returns:
    a one value non-dimensional array
    '''
    pass

<details><summary>
Click here for solution...
</summary>
```
def gradient_one_coefficient_one_row(y_i, x_i, beta_guess, coeff_index):
    return -1 * x_i[coeff_index] * (y_i - np.dot(x_i, beta_guess))
```
</details>

In [ ]:
# guess betas
beta_guess = np.array([1, 1]).reshape(-1, 1)

## try it for one row of data
row = 1
x_i = X[row]
y_i = y[row]

grad_Beta0 = gradient_one_coefficient_one_row(y_i, x_i, beta_guess, 0)
grad_Beta1 = gradient_one_coefficient_one_row(y_i, x_i, beta_guess, 1)
print("Gradient of cost function wrt Beta0: ", grad_Beta0)
print("Gradient of cost function wrt Beta1: ", grad_Beta1)

Let's leverage numpy to calculate the gradients for both coefficients for all rows

In [ ]:
def gradients_coefficients(y, X, beta_guess):
    return -1 * X * (y - np.dot(X, beta_guess))

In [ ]:
beta_guess = np.array([1, 1]).reshape(-1, 1)

gradients = gradients_coefficients(y, X, beta_guess)
gradients

Using **batch** gradient descent, we're going to accumulate the gradient for all rows of data first, and then make an update.  So we don't care about the gradient for each row - rather the accumulated gradient across all rows.  As each accumulated gradient will be applied to each coefficient, we want to return it as the same shape as the coefficients (a column vector -- note the .reshape(-1, 1) )

In [ ]:
def gradients_coefficients_accumulated(y, X, beta_guess):
    gradients = -1 * X * (y - np.dot(X, beta_guess))
    return gradients.sum(axis=0).reshape(-1, 1)

In [ ]:
beta_guess = np.array([1, 1]).reshape(-1, 1)

gradients_accum = gradients_coefficients_accumulated(y, X, beta_guess)
gradients_accum

#### Now, check out this serious numpy magic:

In [ ]:
def mse_grad_func(X, beta_guess, y):
    return np.dot(X.T, np.dot(X, beta_guess)-y)

In [ ]:
beta_guess = np.array([1, 1]).reshape(-1, 1)

gradients_accum = mse_grad_func(X, beta_guess, y)
gradients_accum

#### 2. Adjust weights/parameters by the gradient of the cost function scaled by the learning rate
Update the betas given the gradient of the cost function that was just computed, scaled by the learning rate. This is gradient DESCENT, so we should subtract our update. 

Write a function that performs a parameter update and returns updated parameters (This is very simple - should be a one liner, use numpy. DO NOT OVERTHINK THIS!). Since our problem is simple, the default learning rate is (relatively) high at 0.02.

In [ ]:
def paramater_update(betas, grad, lr=0.02):
    return betas - lr * grad 

<details><summary>
Click here for solution...
</summary>
```
def paramater_update(betas, grad, lr=0.02):
    return betas - lr * grad
```
</details>

**3. Compute objective function, a.k.a the cost function**  
We've done this. We are doing a linear regression, so we will use the mean squared error cost function. 
$$ J(\theta) = \frac{1}{N}\sum_{i=1}^N (y_i - h_\theta(x_i))^2 $$

**4. Check for convergence**  
More later.

<br>
<br>

**Using the gradient and the parameter update functions, you can perform simple gradient descent.**  
Start with guesses for beta

In [ ]:
beta_guess = np.ones((2,1))
beta_guess

Set the iteration count

In [ ]:
i = 0

Execute the cell below as many times as desired.

In [ ]:
i += 1
print('Iteration {}'.format(i))
grad = mse_grad_func(X, beta_guess, y)
print('The gradient is: {0:0.2f}, {1:0.2f}'.format(grad[0][0], grad[1][0]))
beta_guess = paramater_update(beta_guess, grad)
print('New values of beta: {0:0.2f}, {1:0.2f}'.format(beta_guess[0][0], beta_guess[1][0]))
print('  Recall true beta: {0:0.2f}, {1:0.2f}'.format(true_betas[0][0], true_betas[1][0]))

### Put it all together to implement simple gradient descent

In [ ]:
def simple_gradient_descent(X, y, beta_guess = np.ones((2,1)), lr = .02, max_iter = 10): 
    for _ in range(max_iter):
        grad = mse_grad_func(X, beta_guess, y)
        beta_guess = paramater_update(beta_guess, grad, lr)
    return beta_guess

In [ ]:
# does it find the right betas?
betas = simple_gradient_descent(X, y, max_iter = 1000)
print("Calculated betas: {}".format(np.around(betas.ravel(), decimals = 2)))
print("      True betas: {}".format(np.around(true_betas.ravel(), decimals = 2)))

### Gradient Descent Convergence Criterion
We did the simplest case of convergence criteria, a set number of iterations. In practice, you would want to use a more sophisticated convergence criterion - i.e. stopping iterations when your result stops significantly improving.

We are updating the ```simple_gradient_descent```function from above into the second block below, but improved it by adding the following stopping criteria in addition to maximum iterations. Feel free to adjust the default paramaters if you don't like the results.

* Change in cost function $ (cost_{old} - cost_{new}) / cost_{old} < \epsilon $

The stopping criteria requires the cost to be calculated, so we will use the ```cost_function``` function

In [ ]:
def cost_function(X, y, beta_guess):
    return 1/X.shape[0] * np.sum((y - np.dot(X, beta_guess))**2)

In [ ]:
def gradient_descent_with_conv(X, y, beta_guess = np.ones((2,1)), lr = .02,
                               max_iter = 10000, epsilon = 0.01): 
    cost_array = []
    for i in range(max_iter):
        cost_new = cost_function(X, y, beta_guess)
        cost_array.append(cost_new)
        if i > 2:
            cost_old = cost_array[-2]
            if abs(cost_old - cost_new)/cost_old < epsilon:
                print('Convergence met at iteration {0}.'.format(i))
                break
        grad = mse_grad_func(X, beta_guess, y)
        beta_guess = paramater_update(beta_guess, grad, lr)
    return beta_guess

In [ ]:
betas = gradient_descent_with_conv(X, y, max_iter=10000, epsilon = 0.01)  # you may need to play with the learning rate, max_iter, and epsilon
print("Calculated betas: {}".format(np.around(betas.ravel(), decimals = 2)))
print("      True betas: {}".format(np.around(true_betas.ravel(), decimals = 2)))

### Appendix

1. More detailed look into [other gradient descent optimization algorithms](https://ruder.io/optimizing-gradient-descent/index.html#batchgradientdescent)

2. A Visualization of a 2D Gradient Descent
![gradient1d](images/gradientdescent_2d.gif)

3. Another visualization of 3D Gradient Descent:
![grad_desc_gif](images/gradient_descent.gif)